<hr style="border-width:2px;border-color:#75DFC1">
<center><h1> Pandas for Data Science</h1></center>
<center><h2> Data processing </h2></center>
<hr style="border-width:2px;border-color:#75DFC1">


> Data processing can be described in 4 essential operations: **filtering**, **merging**, **ordering** and **grouping**.
>
> The DataFrame class has risen to prominence in the field of data manipulation precisely because it often only requires repetition or combination of these four operations.
>
> In this lesson, you will learn how to use these 4 operations of data processing.
>


* Before starting this notebook, **run the following cell** in order to retrieve the work done in the previous exercises.

In [1]:
### Imports ###

import pandas as pd

In [2]:
# Data import
transactions = pd.read_csv("transactions.csv", sep =',', index_col = "transaction_id")

In [3]:
transactions.head(10)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1.0,1,-5,-772.0,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5.0,3,-5,-1497.0,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6.0,5,-2,-791.0,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11.0,6,-3,-1363.0,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6.0,5,-2,-791.0,166.110,-1748.110,TeleShop
97439039119,272357,23-02-2014,8.0,3,-2,-824.0,173.040,-1821.040,TeleShop
45649838090,273667,22-02-2014,11.0,6,-1,-1450.0,152.250,-1602.250,e-Shop
22643667930,271489,22-02-2014,12.0,6,-1,-1225.0,128.625,-1353.625,TeleShop
79792372943,275108,22-02-2014,3.0,1,-3,-908.0,286.020,-3010.020,MBR


In [4]:
# Removal of duplicates
transactions = transactions.drop_duplicates(keep = 'first')

In [5]:
# Changing the names of the columns
new_names =  {'Store_type' : 'store_type',
              'Qty'        : 'qty',
              'Rate'       : 'rate',
              'Tax'        : 'tax'}

transactions = transactions.rename(new_names, axis = 1)
transactions.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1.0,1,-5,-772.0,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5.0,3,-5,-1497.0,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6.0,5,-2,-791.0,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11.0,6,-3,-1363.0,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6.0,5,-2,-791.0,166.110,-1748.110,TeleShop


In [6]:
transactions['prod_subcat_code'].isnull().sum()

np.int64(32)

In [7]:
#  We replace the NAs in 'prod_subcat_code' by -1
transactions['prod_subcat_code'] = transactions['prod_subcat_code'].fillna(-1).astype(int)

In [8]:
transactions['prod_subcat_code'].isnull().sum()

np.int64(0)

In [9]:
transactions['store_type'].mode()

0    e-Shop
Name: store_type, dtype: object

In [10]:
# We compute the mode of 'store_type'
store_type_mode = transactions['store_type'].mode()

# We replace the NAs of 'store_type' by its mode
transactions['store_type'] = transactions['store_type'].fillna(store_type_mode[0])

In [11]:
# Removal of rows where 'rate', 'tax' and 'total_amt' are all NAs
transactions = transactions.dropna(axis = 0, how = 'all', subset = ['rate', 'tax', 'total_amt'])

In [12]:
transactions.shape

(22919, 9)


## 1. Filtering a `DataFrame` with binary operators.

> Filtering involves **selecting** a subset of **rows** in a DataFrame based on a given **condition**. This process aligns with what we previously referred to as conditional indexing, though in database management, "filtering" is the preferred term.
>
> It's worth noting that we cannot employ the logical operators `and` and `or` for filtering with multiple conditions. These operators can introduce **ambiguities** that pandas cannot effectively manage for filtering.
>
> In this context, the appropriate operators for filtering based on multiple conditions are the binary **operators**:
>
>> - The 'and' operator: `&`.
>>
>> - The 'or' operator: `|`.
>>
>> - The 'not' operator: `-`.

> While these operators share similarities with logical operators, their evaluation methods are not identical.

### The 'and' operator: `&`.

> The `&` operator is employed to filter a `DataFrame` based on multiple conditions that need to be satisfied **simultaneously**.
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span>
>
> Let's consider the `DataFrame` `df` that provides information about apartments in Paris:
>
> |       | district         | year | surface |
> |-------|------------------|------|---------|
> | **0** | 'Champs-Elysées' | 1979 | 70      |
> | **1** | 'Europe'         | 1850 | 110     |
> | **2** | 'Père-Lachaise'  | 1935 | 55      |
> | **3** | 'Bercy'          | 1991 | 30      |
>
> If our objective is to locate an apartment from the year 1979 **and** with a surface area exceeding 60 square meters, we can filter the rows in `df` using the following code:
>
> ``` py
>
> # Filtering of the DataFrame on the 2 previous conditions
> print(df[(df['year'] == 1979) & (df['surface']> 60)])
> ```
>
> ```
> >>>         district  year  surface
> >>> 0  Champs-Elysées  1979       70
> ```
>
> The conditions must be written **between parentheses** to eliminate any ambiguity on the **order of evaluation** of the conditions. Indeed, if the conditions are not properly separated, we will get the following error:
>
> ``` python
> print(df[df['year'] == 1979 & df['surface']> 60])
> ```
> ```
> >>> ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
> ```

### The 'or' operator: `|`.

> The operator `|` is used to filter a `DataFrame` on several conditions of which **one at least** must be verified.
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span>
>
> Consider the same `DataFrame` `df`:
>
> |       | district         | year | surface |
> |-------|------------------|------|---------|
> | **0** | 'Champs-Elysées' | 1979 | 70      |
> | **1** | 'Europe'         | 1850 | 110     |
> | **2** | 'Père-Lachaise'  | 1935 | 55      |
> | **3** | 'Bercy'          | 1991 | 30      |
>
> If we want to find an apartment that dates after 1900 **or** is located in the Père-Lachaise district, we can filter the lines of `df` with the following code:
>
> ``` py
>
> # Filtering of the DataFrame on the 2 previous conditions
> print(df[(df['year']> 1900) | (df['district'] == 'Père-Lachaise')])
> ```
> ```
> >>>          district  year  surface
> >>> 0  Champs-Elysées  1979       70
> >>> 2  Père-Lachaise   1935       55
> >>> 3  Bercy           1991       30
> ```
>
>
### The 'not' operator: `-`.
>
>The `-` operator allows us to filter a `DataFrame` based on a conditions, of which **the negative** must be satisfied.
>
><span style="color:#09b038; text-decoration : underline"> Example: </span>
>
>Let's consider the same `DataFrame` `df`:
>
>|       | district         | year | surface |
>|-------|------------------|------|---------|
>| **0** | 'Champs-Elysées' | 1979 | 70      |
>| **1** | 'Europe'         | 1850 | 110     |
>| **2** | 'Père-Lachaise'  | 1935 | 55      |
>| **3** | 'Bercy'          | 1991 | 30      |
>
>If we want to find an apartment not located in Bercy district, we can filter `df` using the following code:
>
> ``` py
> # Filtering of the DataFrame on the the negation of a condition
> print(df[-(df['district'] == 'Bercy')])
> ```
> ```
> >>> district year surface
> >>> 0 Champs-Elysées 1979 70
> >>> 1 Europe 1850 110
> >>> 2 Père-Lachaise 1935 55
> ```
>
>
>


* **(a)** Display the first 5 lines of the `transactions` `DataFrame`.

* **(b)** Create a new `DataFrame` named **`e_shop`** from **`transactions`**. Include only transactions that were carried out in stores of type **`'e-Shop'`** with a total amount greater than 5000. This selection involves the `'store_type'` and `'total_amt'` columns.

* **(c)** Similarly, create a `DataFrame` named **``teleshop``** which contains the transactions made in stores of type **`'TeleShop'`** with a total amount of more than 5000.

* **(d)** Determine which type of store, either `'e-Shop'` or `'TeleShop'`, has the highest number of transactions with amounts exceeding € 5,000.


In [13]:
#a Display the first 5 lines of the `transactions` `DataFrame`.
transactions.head(5)

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772.0,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497.0,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363.0,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop


In [14]:
#b Create a new `DataFrame` named **`e_shop`** from **`transactions`**. 
# Include only transactions that were carried out in stores of type **`'e-Shop'`**
#  with a total amount greater than 5000. This selection involves the `'store_type'` and `'total_amt'` columns.
e_shop = transactions[(transactions['total_amt']>5000) & (transactions['store_type']=='e-Shop')][['store_type','total_amt']]
e_shop.head()

,store_type,total_amt
transaction_id,,
29258453508,e-Shop,8270.925
22461440770,e-Shop,5856.500
7719443790,e-Shop,5016.700
2663044947,e-Shop,7956.000
13264686077,e-Shop,5202.340


In [15]:
#c Similarly, create a `DataFrame` named **``teleshop``**
#  which contains the transactions made in stores of type **`'TeleShop'`** with a total amount of more than 5000.
teleshop = transactions[(transactions['total_amt']>5000) & (transactions['store_type']=='TeleShop')]
teleshop

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
28396777609,269101,19-02-2014,4,1,4,1386.0,582.120,6126.120,TeleShop
90740728864,268229,19-02-2014,4,2,5,984.0,516.600,5436.600,TeleShop
58956348529,272484,18-02-2014,4,1,4,1443.0,606.060,6378.060,TeleShop
75980673312,270678,17-02-2014,7,5,4,1377.0,578.340,6086.340,TeleShop
35243275448,273075,15-02-2014,2,6,4,1168.0,490.560,5162.560,TeleShop
...,...,...,...,...,...,...,...,...,...
18451702646,268363,2-2-2011,12,5,5,1046.0,549.150,5779.150,TeleShop
11030274761,268688,30-01-2011,4,1,5,1351.0,709.275,7464.275,TeleShop
70883882641,273674,29-01-2011,11,5,5,1206.0,633.150,6663.150,TeleShop


In [16]:
#d Determine which type of store, either `'e-Shop'` or `'TeleShop'`, has the highest number of transactions with amounts exceeding € 5,000.
print('Number of transactions over 5000€ for e-shop :', len(e_shop['total_amt']))
print('Number of transactions over 5000€ for teleshop :', len(teleshop['total_amt']))

Number of transactions over 5000€ for e-shop : 1185
Number of transactions over 5000€ for teleshop : 532


* **(e)** Load the data from the files `'customer.csv'` and `'prod_cat_info.csv'` into two separate DataFrames named **`customer`** and **`prod_cat_info`**.

* **(f)** In the `customer` DataFrame, there are two missing values in both the `Gender` and `city_code` columns. Fill these missing values with the mode of their respective columns using the `fillna` method along with the `mode` method.

In [ ]:
#e Load the data from the files `'customer.csv'` and `'prod_cat_info.csv'` 
# into two separate DataFrames named **`customer`** and **`prod_cat_info`**.
customer = pd.read_csv('customer.csv')
prod_cat_info = pd.read_csv('prod_cat_info.csv')

In [37]:
customer.isna().sum()

customer_Id    0
DOB            0
Gender         0
city_code      2
dtype: int64

In [38]:
#f In the `customer` DataFrame, there are two missing values in both the `Gender` and `city_code` columns.
#  Fill these missing values with the mode of their respective columns using the `fillna` method along with the `mode` method.
customer['Gender'] = customer['Gender'].fillna(customer['Gender'].mode()[0])
customer['city_code'] = customer['city_code'].fillna(customer['city_code'].mode()[0])



## 2. Joining `Dataframes`: `concat` function and `merge` method.

### Concatenation of `DataFrames` with `concat`

> The `concat` function from the `pandas` module enables the concatenation of multiple `DataFrames`, either by stacking them vertically or by aligning them side by side.
>
>
> ```python
> pandas.concat(objs, axis, ...)
> ```
>
> - The `objs` parameter takes a list of `DataFrames` to concatenate.
> - The `axis` parameter specifies whether the concatenation should be vertical (`axis = 0`) or horizontal (`axis = 1`).
>
>
> <br/>
> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_concat_en.png', style = "height:400px">
> <br/>
>
> When the number of rows or columns of the `DataFrames` does not match, the` concat` function fills the empty cells with `NaN`, as shown in the illustration below.
>
> <br/>
> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_concat_none_en.png', style = "height:400px">
> <br/>
>

* **(a)** Divide the **columns** of the **`transactions`** `DataFrame` in half, placing the first half in a `DataFrame` named **`part_1`**, and the second half in a `DataFrame` named **`part_2`**.

* **(b)** Reconstruct the **`transactions`** data in a `DataFrame` named **`union`** by horizontally concatenating **`part_1`** and **`part_2`**.

* **(c)** What happens if we concatenate `part_1` and `part_2` with the argument **`axis = 0`** filled in?

In [ ]:
#Divide the **columns** of the **`transactions`** `DataFrame` in half, placing the first half in a `DataFrame` named **`part_1`**,
#  and the second half in a `DataFrame` named **`part_2`**
columns = transactions.columns
part_1  = transactions[columns[:5]]
part_1.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty
transaction_id,,,,,
80712190438,270351,28-02-2014,1,1,-5
29258453508,270384,27-02-2014,5,3,-5
51750724947,273420,24-02-2014,6,5,-2
93274880719,271509,24-02-2014,11,6,-3
51750724947,273420,23-02-2014,6,5,-2


In [50]:
part_2  = transactions[columns[5:]]
part_2.head()

,rate,tax,total_amt,store_type
transaction_id,,,,
80712190438,-772.0,405.300,-4265.300,e-Shop
29258453508,-1497.0,785.925,-8270.925,e-Shop
51750724947,-791.0,166.110,-1748.110,TeleShop
93274880719,-1363.0,429.345,-4518.345,e-Shop
51750724947,-791.0,166.110,-1748.110,TeleShop


In [ ]:
#b Reconstruct the **`transactions`** data in a `DataFrame` named **`union`** by horizontally concatenating **`part_1`** and **`part_2`**
new = pd.concat([part_1, part_2], axis=1)
new.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772.0,405.300,-4265.300,e-Shop
29258453508,270384,27-02-2014,5,3,-5,-1497.0,785.925,-8270.925,e-Shop
51750724947,273420,24-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop
93274880719,271509,24-02-2014,11,6,-3,-1363.0,429.345,-4518.345,e-Shop
51750724947,273420,23-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop


In [ ]:
#c What happens if we concatenate `part_1` and `part_2` with the argument **`axis = 0`** filled in?

new1 = pd.concat([part_1, part_2], axis=0)
new1.head()
# If we were to concatenate by filling in the argument "axis = 0", we would obtain a DataFrame where half of
# the valuers are NAs
#
# This is due to the fact that the argument 'axis = 0' forces the pd.concat function to create new ROWS
# in part_1 but it cannot fill them correctly since part_1 and part_2 have no columns in common.


,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
80712190438,270351.0,28-02-2014,1.0,1.0,-5.0,NaN,NaN,NaN,NaN
29258453508,270384.0,27-02-2014,5.0,3.0,-5.0,NaN,NaN,NaN,NaN
51750724947,273420.0,24-02-2014,6.0,5.0,-2.0,NaN,NaN,NaN,NaN
93274880719,271509.0,24-02-2014,11.0,6.0,-3.0,NaN,NaN,NaN,NaN
51750724947,273420.0,23-02-2014,6.0,5.0,-2.0,NaN,NaN,NaN,NaN


In [56]:
new1.tail()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type
transaction_id,,,,,,,,,
94340757522,NaN,NaN,NaN,NaN,NaN,1264.0,132.720,1396.720,e-Shop
89780862956,NaN,NaN,NaN,NaN,NaN,677.0,71.085,748.085,e-Shop
85115299378,NaN,NaN,NaN,NaN,NaN,1052.0,441.840,4649.840,MBR
72870271171,NaN,NaN,NaN,NaN,NaN,1142.0,359.730,3785.730,TeleShop
77960931771,NaN,NaN,NaN,NaN,NaN,447.0,46.935,493.935,TeleShop



### Merging `DataFrames` with the `merge` method

>Two `DataFrames` can be combined through a process called merging, provided they share a common column. This is accomplished using the `merge` method of the `DataFrame` class, which has the following syntax:
>
>>```python
>>merge(right, on, how, ...)
>>```
>
>- The **`right`** parameter represents the `DataFrame` to be merged with the one calling the method.
>
>- The **`on`** parameter specifies the names of the columns in both `DataFrames` that will serve as references for the merge. These columns must be shared between the two.
>
>- The **`how`** parameter lets you choose the type of join to perform when merging the `DataFrames`. The available values for this parameter are based on SQL-style joins.

The `how` parameter can take one of four values: `'inner'`, `'outer'`, `'left'`, or `'right'`. These will be illustrated using two sample `DataFrames` named `Persons` and `Vehicles` below:
>
>
> | Name | Car |
> | ---------- | ------------ |
> | Lila | Twingo |
> | Tiago | Clio |
> | Berenice | C4 Cactus |
> | Joseph | Twingo |
> | Kader | Swift |
> | Romy | Scenic |
>
> | Car | Price |
> | ----------- | ------- |
> | Twingo | 11000 |
> | Swift | 14500 |
> | C4 Cactus | 23000 |
> | Clio | 16000 |
> | Prius | 30000 |
>
>
>>- **`'inner'`**: The inner join returns the rows where the values in the common columns are **found in both `DataFrames`**. It's important to note that this type of join may result in the loss of many entries. However, the inner join does not generate **NAs**.
>>
>>
>> The result of the inner join `Persons.merge(right = Vehicles, on = 'Car', how = 'inner')` is shown below:
>>
>> <br>
>> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_join_inner_en.png' style="height:700px">
>> <br>
>> <br>
>>
>> - **`'outer'`**: The outer join combines the two `DataFrames` **in their entirety**. No row will be deleted. This method can generate **a lot of NAs**.
>>
>> The result of the outer join `Persons.merge(right = Vehicles, on = 'Car', how = 'outer')` is shown below:
>>
>> <br>
>> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_join_outer_en.png' style="height:700px">
>> <br>
>> <br>
>>
>>- **`'left'`**: The left join returns **all the rows** of the `DataFrame` on the **left** (i.e. the one calling the method), and complements them with the rows of the second `DataFrame` that match according to the values of the common column. This is the **default value for the `how`** parameter.
>>
>> The result of the left join `Persons.merge(right = Vehicles, on = 'Car', how = 'left')` is shown below:
>>
>> <br>
>> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_join_left_en.png' style="height:700px">
>> <br>
>> <br>
>>
>> - **`'right'`**: The right join returns **all the rows** of the `DataFrame` on the **right**, and complete them with the rows of the left `DataFrame` which coincide according to the values of the common column.
>>
>> The result of the right join `Persons.merge(right = Vehicles, on = 'Car', how = 'right')` is shown below:
>>
>> <br>
>> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_join_right_en.png' style="height:700px">
>> <br>
>> <br>
>
>> Performing a left join, right join, or outer join followed by a `dropna(how = 'any')` is essentially equivalent to an inner join.
>
> The `customer` `DataFrame` holds customer information under the `'cust_id'` column, which corresponds to the customer ID in `transactions`.
>
> The `'customer_Id'` column in the `customer` `DataFrame` will be used as the linking column for joining `transactions` and `customer`. This will enhance the `transactions` `DataFrame` with supplementary information.
>
> *(d)* Rename the **`'customer_Id'`** column in the **`customer`** `DataFrame` to **`'cust_id'`** using the `rename` method along with a dictionary.
>
> *(e)* Utilize the `merge` method to execute a **left join** between the `DataFrames` **`transactions`** and **`customer`** based on the `'cust_id'` column. Name the resulting `DataFrame` **`fusion`**.
>
> *(f)* Did the merging process result in any NAs?
>
> *(g)* Display the initial rows of **`fusion`**. What are the newly introduced columns?



In [58]:
transactions.columns

Index(['cust_id', 'tran_date', 'prod_subcat_code', 'prod_cat_code', 'qty',
       'rate', 'tax', 'total_amt', 'store_type'],
      dtype='object')

In [57]:
customer.columns

Index(['customer_Id', 'DOB', 'Gender', 'city_code'], dtype='object')

In [59]:
#d Rename the **`'customer_Id'`** column in the **`customer`** 
# `DataFrame` to **`'cust_id'`** using the `rename` method along with a dictionary.
customer = customer.rename({'customer_Id': 'cust_id'}, axis = 1)
customer.columns

Index(['cust_id', 'DOB', 'Gender', 'city_code'], dtype='object')

In [62]:
#e Utilize the `merge` method to execute a **left join**
#  between the `DataFrames` **`transactions`** and **`customer`** based on the `'cust_id'` column. 
# Name the resulting `DataFrame` **`fusion`**.
fusion = transactions.merge(right=customer, on='cust_id', how='left')


In [61]:
#f Did the merging process result in any NAs?
fusion.isna().sum()

cust_id             0
tran_date           0
prod_subcat_code    0
prod_cat_code       0
qty                 0
rate                0
tax                 0
total_amt           0
store_type          0
DOB                 0
Gender              0
city_code           0
dtype: int64

In [63]:
#g Display the initial rows of **`fusion`**. What are the newly introduced columns?
fusion.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,DOB,Gender,city_code
0,270351,28-02-2014,1,1,-5,-772.0,405.300,-4265.300,e-Shop,26-09-1981,M,5.0
1,270384,27-02-2014,5,3,-5,-1497.0,785.925,-8270.925,e-Shop,11-05-1973,F,8.0
2,273420,24-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop,27-07-1992,M,8.0
3,271509,24-02-2014,11,6,-3,-1363.0,429.345,-4518.345,e-Shop,08-06-1981,M,3.0
4,273420,23-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop,27-07-1992,M,8.0



> The merging went well and produced no NaNs. However, the index of the `DataFrame` is no longer the column **`transaction_id'`** and has been reset with the default index (`0`,` 1`, `2` , ...).
>
> You can re-define the index of a `DataFrame` using the **`set_index`** method.
>
> This method can take as argument:
>> * The **name** of a column to use as indexing.
>> * A `numpy` `array` or `pandas` `Series`  with the same number of rows as the `DataFrame` calling the method.
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span><br>
>
> Let `df` be the following `DataFrame`:
>
> |       | Name     | Car       |
> |-------|----------|-----------|
> | **0** | Lila     | Twingo    |
> | **1** | Tiago    | Clio      |
> | **2** | Berenice | C4 Cactus |
> | **3** | Joseph   | Twingo    |
> | **4** | Kader    | Swift     |
> | **5** | Romy     | Scenic    |
>
> We can set the column `'Name'` as the new index:
>
> ```python
> df = df.set_index('Name')
> ```
>
> This will produce the following `DataFrame`:
>
> | <br><br><br> **Name** | Car       |
> |:----------------------|:----------|
> | **Lila**              | Twingo    |
> | **Tiago**             | Clio      |
> | **Berenice**          | C4 Cactus |
> | **Joseph**            | Twingo    |
> | **Kader**             | Swift     |
> | **Romy**              | Scenic    |
>
> We can also define the index from an `array`, from a `Series`, etc:
>
> ```python
> # New index to use
> new_index = ['10000' + str(i) for i in range(6)]
> print(new_index)
> >>> ['100000', '100001', '100002', '100003', '100004', '100005']
>
> # Using an array or a Series is equivalent
> index_array = np.array(new_index)
> index_series = pd.Series(new_index)
>
>
> df = df.set_index(index_array)
> df = df.set_index(index_series)
> ```
>
> This will produce the following `DataFrame`:
>
> |            | Name     | Car       |
> |-----------:|:---------|:----------|
> | **100000** | Lila     | Twingo    |
> | **100001** | Tiago    | Clio      |
> | **100002** | Berenice | C4 Cactus |
> | **100003** | Joseph   | Twingo    |
> | **100004** | Kader    | Swift     |
> | **100005** | Romy     | Scenic    |

> To return to the default numeric indexing, we use the **`reset_index`** method of the `DataFrame`:
>
> ```python
> df = df.reset_index()
> ```
>
> The indexing column that was used **is not deleted**. A new column will be created containing the old index:
>
> |       | index  | Name | Car |
> |------:|-------:|:---------|:----------|
> | **0** | 100000 | Lila | Twingo |
> | **1** | 100001 | Tiago | Clio |
> | **2** | 100002 | Berenice | C4 Cactus |
> | **3** | 100003 | Joseph | Twingo |
> | **4** | 100004 | Kader | Swift |
> | **5** | 100005 | Romy | Scenic |
>

Merging `transactions` and` customer` removed the index of `transactions`.

The index of a `DataFrame` can be retrieved using its `.index` attribute.

* **(h)** Take the index from `transactions` and use it to index `fusion`.



In [ ]:
# fusion = fusion.set_index(transactions.index)

In [67]:
fusion.set_index(transactions.index, inplace=True)
fusion.head()

,cust_id,tran_date,prod_subcat_code,prod_cat_code,qty,rate,tax,total_amt,store_type,DOB,Gender,city_code
transaction_id,,,,,,,,,,,,
80712190438,270351,28-02-2014,1,1,-5,-772.0,405.300,-4265.300,e-Shop,26-09-1981,M,5.0
29258453508,270384,27-02-2014,5,3,-5,-1497.0,785.925,-8270.925,e-Shop,11-05-1973,F,8.0
51750724947,273420,24-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop,27-07-1992,M,8.0
93274880719,271509,24-02-2014,11,6,-3,-1363.0,429.345,-4518.345,e-Shop,08-06-1981,M,3.0
51750724947,273420,23-02-2014,6,5,-2,-791.0,166.110,-1748.110,TeleShop,27-07-1992,M,8.0



## 3. Sort and order the values of a `DataFrame`: `sort_values` and `sort_index` methods.


> The `sort_values` method allows you to sort the rows of a `DataFrame` according to the values of one or more columns.
>
> The header of this method is as follows:
>
> `sort_values(by, ascending, ...)`
>
>> - The `by` parameter allows you to specify on which column(s) the sort is performed.
>>
>> - The `ascending` parameter is a boolean value (`True` or `False`) determining whether the sorting order is ascending or descending. By default this parameter is set to `True`.
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span><br>
>
> Consider the `DataFrame` `df` describing students:
>
>
> | Name     | Grade | Bonus points |
> |----------|-------|--------------|
> | 'Amelie' | A     | 1            |
> | 'Marin'  | F     | 1            |
> | 'Pierre' | A     | 2            |
> | 'Zoe'    | C     | 1            |
>
> First of all, we will sort the rows on a single column, for example the column `'Bonus Points'`:
>
> ```python
> # We sort the DataFrame df on the column 'Bonus Points'
> df_sorted = df.sort_values(by ='Bonus Points', ascending = True)
> ```
>
> We obtain the following result:
>
> <br/>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_sort_values_en.png' style="height:400px">
> <br>
>
> The rows of the `DataFrame` `df_sorted` are therefore sorted in **ascending order** of the **`'Bonus points'`** column. However, if we look at the column `'Grade'`, we see that it is not sorted alphabetically for the common values of `'Bonus Points'`.
>
> This can be remedied by also sorting by the `'Grade'` column:
>
>
> ```python
> # We first sort the DataFrame df by the column 'Bonus Points' then in case of equality, by the column 'Grade'.
> df_sorted = df.sort_values(by = ['Bonus Points', 'Grade'], ascending = True)
> ```
>
> <br/>
> We obtain the following result:
>
> <br><br>
>
> <img src='https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_sort_values_2_en.png' style="height:600px">
>
> <br>
>
> <br/>
>
> The **`sort_index`** method is used to sort a `DataFrame` based on its index.
>
> When the index is the default numerical one, this method may not offer significant advantages. However, it can frequently be utilized in conjunction with the `set_index` method of a `DataFrame`, as we've recently covered.
>
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span><br>
>
> ```py
> # We define the column 'Grade' as the index of df
> df = df.set_index('Grade')
>
> # We sort the DataFrame df according to its index
> df = df.sort_index()
>
> ```
>
> This produces the following `DataFrame`:
>
> | <br><br><br> **Grade**| Name    | Bonus points |
> |-----------------------|---------|--------------|
> | **A**                 |'Amelie' | 1            |
> | **A**                 |'Peter'  | 2            |
> | **C**                 |'Zoe'    | 1            |
> | **F**                 |'Sailor' | 1            |
>             
>
>Consider the two following `DataFrames` containing boat rental data.
>
>Below are the `boats` `DataFrame`:
>
>
>|       | boat_name | color  | reservation_number | n_reservations |
>|------:|:----------|:-------|-------------------:|---------------:|
>| **0** | Julia     | blue   | 2                  | 34             |
>| **1** | Siren     | green  | 3                  | 10             |
>| **2** | Sea Sons  | red    | 6                  | 20             |
>| **3** | Hercules  | blue   | 1                  | 41             |
>| **4** | Cesar     | yellow | 4                  | 12             |
>| **5** | Minerva   | green  | 5                  | 16             |
>
>
>And the `clients` `DataFrame`:
>
>|       | client_id | client_name | reservation_id |
>|------:|----------:|:------------|---------------:|
>| **0** | 91        | Marie       | 1              |
>| **1** | 154       | Anna        | 2              |
>| **2** | 124       | Yann        | 3              |
>| **3** | 320       | Lea         | 7              |
>| **4** | 87        | Marc        | 9              |
>| **5** | 22        | Yassine     | 10             |

* **(a)** Run the following cell to instantiate these `DataFrames`.



In [94]:
# Definition of the data dictionnaries
data_boats = {'boat_name'         : ['Julia', 'Siren', 'Sea Sons', 'Hercules', 'Cesar', 'Minerva'], 
              'color'             : ['blue', 'green', 'red', 'blue', 'yellow', 'green'],
              'reservation_number': [2, 3, 6, 1, 4, 5],
              'n_reservations'    : [34, 10, 20, 41, 12, 16]}

data_clients = {'client_id'     : [91, 154, 124, 320, 87, 22], 
                'client_name'   : ['Marie', 'Anna', 'Yann', 'Lea', 'Marc', 'Yassine'],
                'reservation_id': [1, 2, 3, 7, 9, 10]}

# Instantiation of the DataFrames
boats = pd.DataFrame(data_boats)
clients = pd.DataFrame(data_clients)


> We want to easily determine which customer has reserved the boats of the `boats` `DataFrame`. To do this, we can simply merge the `DataFrames`.

* **(b)** Rename the `'reservation_number'` column from `boats` to `'reservation_id'` using the `rename` method.


* **(c)** In a `DataFrame` named **`boats_clients`**, perform the **left join** between `boats` (left) and `clients` (right).


* **(d)** Set the column `'boat_name'` as index of the `boats_clients` `DataFrame`.


* **(e)** Using the `loc` method, find who reserved the boats `'Julia'` and `'Siren'`.


* **(f)** Using the `isna` method applied to the `client_name` column, determine the boats that have not been reserved.


* **(g)** The number of times a boat has been reserved so far is indicated by the column `'n_reservations'`. Using the **`sort_values`** method, determine the name of the customer who reserved the **blue** boat with the most reservations to date.



In [95]:
#b Rename the `'reservation_number'` column from `boats` to `'reservation_id'` using the `rename` method
boats = boats.rename({'reservation_number': 'reservation_id'}, axis=1)
boats.columns

Index(['boat_name', 'color', 'reservation_id', 'n_reservations'], dtype='object')

In [96]:
clients.columns

Index(['client_id', 'client_name', 'reservation_id'], dtype='object')

In [97]:
#c In a `DataFrame` named **`boats_clients`**, perform the **left join** between `boats` (left) and `clients` (right).
boats_clients = boats.merge(right=clients, on='reservation_id', how='left')
boats_clients.head()

,boat_name,color,reservation_id,n_reservations,client_id,client_name
0,Julia,blue,2,34,154.0,Anna
1,Siren,green,3,10,124.0,Yann
2,Sea Sons,red,6,20,NaN,NaN
3,Hercules,blue,1,41,91.0,Marie
4,Cesar,yellow,4,12,NaN,NaN


In [98]:
boats.head()

,boat_name,color,reservation_id,n_reservations
0,Julia,blue,2,34
1,Siren,green,3,10
2,Sea Sons,red,6,20
3,Hercules,blue,1,41
4,Cesar,yellow,4,12


In [99]:
#d Set the column `'boat_name'` as index of the `boats_clients` `DataFrame`.
boats_clients  = boats_clients.set_index('boat_name')
boats_clients.head()

,color,reservation_id,n_reservations,client_id,client_name
boat_name,,,,,
Julia,blue,2,34,154.0,Anna
Siren,green,3,10,124.0,Yann
Sea Sons,red,6,20,NaN,NaN
Hercules,blue,1,41,91.0,Marie
Cesar,yellow,4,12,NaN,NaN


In [100]:
#e Using the `loc` method, find who reserved the boats `'Julia'` and `'Siren'`.
boats_clients.loc['Julia']

color              blue
reservation_id        2
n_reservations       34
client_id         154.0
client_name        Anna
Name: Julia, dtype: object

In [101]:
boats_clients.loc['Siren']

color             green
reservation_id        3
n_reservations       10
client_id         124.0
client_name        Yann
Name: Siren, dtype: object

In [102]:
#f Using the `isna` method applied to the `client_name` column, determine the boats that have not been reserved.
boats_clients['client_name'].isna()

boat_name
Julia       False
Siren       False
Sea Sons     True
Hercules    False
Cesar        True
Minerva      True
Name: client_name, dtype: bool

In [ ]:
# Lösung
# # Which boats have not been reserved?
# boats_not_reserved = boats_clients[boats_clients['client_name'].isna()]
# print("The boats which have not been reserved are:", [boat for boat in boats_not_reserved.index])

In [ ]:
#g The number of times a boat has been reserved so far is indicated by the column `'n_reservations'`.
#  Using the **`sort_values`** method, determine the name of the customer who reserved the **blue** boat with the most reservations to date.
boats_clients[boats_clients['color']=='blue'].sort_values(by='n_reservations')

# # Which client reserved the BLUE boat with the MOST reservations to date?
# most_reserved_blue_boat = boats_clients[boats_clients['color']=='blue'].sort_values(by = 'n_reservations', ascending = False).iloc[0]
# print("The customer who has booked the blue boat with the most reservations is :", most_reserved_blue_boat['client_name'])

,color,reservation_id,n_reservations,client_id,client_name
boat_name,,,,,
Julia,blue,2,34,154.0,Anna
Hercules,blue,1,41,91.0,Marie



# 4. Grouping the elements of a `DataFrame`: `groupby`, `agg` and `crosstab` methods.

> The **`groupby`** method allows you to **group the rows** of a `DataFrame` which share a **common** value on a given column.
>
> **This method does not return a `DataFrame`.** The object returned by the `groupby` method is an object of the **`DataFrameGroupBy`** class.
>
>
> This class is used to perform operations such as calculating statistics (sum, average, maximum, etc.) for each category of the column on which the rows are grouped.
>
> The general structure of a **`groupby` operation** is as follows:
>
>> * **Split** the data.
>> * **Apply** a function.
>> * **Combine** the results.
>
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span><br>
>
> Assuming all the boats in the `boats` `DataFrame` are identical in terms of age, we aim to determine if the color of a boat impacts its reservation frequency. To do this, we will calculate the average number of reservations per boat for each color.
>
> The process involves:
>
>> * **Splitting** the boats based on color.
>> * **Applying** the **`mean`** function to determine the average number of reservations.
>> * **Combining** the outcomes in a `DataFrame` for easy comparison.
>
> To achieve this, we can utilize the **`groupby`** method followed by the **`mean`** method, resulting in:
>
> <br>
>
> <img src= 'https://assets-datascientest.s3-eu-west-1.amazonaws.com/train/pd_groupby_en.png' style="height:350px">
>
> <br>
>
> All the usual statistical methods (`count`,` mean`, `max`, etc.) can be used as a suffix of the `groupby` method. These will only be applied on columns of compatible type.
>
> For each column, you can specify which function should be used in the **Apply** step of a `groupby` operation. To do this, we use the **`agg`** method of the `DataFrameGroupBy` class. This involves providing a **dictionary** where each **key** represents the **name** of a column, and the corresponding **value** indicates the **function** to be applied.
>
>
> <span style="color:#09b038; text-decoration : underline"> Example: </span><br>
>
> Let us go back to the `transactions` `DataFrame`:
>
> | transaction_id | cust_id | tran_date  | prod_subcat_code | prod_cat_code | qty | rate | tax     | total_amt | store_type |
> |---------------:|--------:|:-----------|-----------------:|--------------:|----:|-----:|--------:|----------:|:-----------|
> | 80712190438    | 270351  | 28-02-2014 | 1                | 1             | -5  |-772  | 405.3   | -4265.3   | e-Shop     |
> | 29258453508    | 270384  | 27-02-2014 | 5                | 3             | -5  |-1497 | 785.925 | -8270.92  | e-Shop     |
> | 51750724947    | 273420  | 24-02-2014 | 6                | 5             | -2  |-791  | 166.11  | -1748.11  | TeleShop   |
> | 93274880719    | 271509  | 24-02-2014 | 11               | 6             | -3  |-1363 | 429.345 | -4518.35  | e-Shop     |
> | 51750724947    | 273420  | 23-02-2014 | 6                | 5             | -2  |-791  | 166.11  | -1748.11  | TeleShop   |
>
>
> We want to determine, **for each customer** (`cust_id`), the **minimum**, **maximum** and the **total amount** spent from the `total_amt` column. We also want to know **how many types of stores** the customer has made a transaction in (`store_type` column).
>
> We can perform these calculations using a **`groupby`** operation:
>
>> * **Split** the transactions by the **customer identifier**.
>> * For the **`total_amt`** column, calculate the minimum (`min`), maximum (`max`) and the sum (`sum`). For the **`store_type`** column, count the **number of unique categories taken**.
>> * **Combine** the results in a `DataFrame`.
>
> To find the number of unique categories taken by the `store_type` column, we will use the following **`lambda`** function:
>
> ```python
> import numpy as np
>
> n_categories = lambda store_type: len(np.unique(store_type))
> ```
>
>> * The `lambda` function must take as argument a **column** and return a **number**.
>> * The **`np.unique`** function determines the unique **categories** that appear in a sequence.
>> * The **`len`** function counts the number of elements in a sequence, i.e. its length.
>
> Thus, this function will allow us to determine the number of unique categories for the `store_type` column.
>
> To apply these functions in the `groupby` operation, we'll use a dictionary whose **keys** are the **columns** to process and the **values** the **functions** to use.
>
> ```python
> functions_to_apply = {
> # Classic statistical methods can be entered with
> # strings
> 'total_amt': ['min', 'max', 'sum'],
> 'store_type': n_categories
>}
> ```
>
> This dictionary can now be fed into the **`agg`** method to perform the `groupby` operation:
>
> ```python
> transactions.groupby('cust_id').agg(functions_to_apply)
> ```
>
> Which produces the following `DataFrame`:
>
> | <br><br><br><br><br> cust_id | total_amt <br><br> min | <br><br> max | <br><br> sum | store_type <br><br> <lambda\> |
> |-----------------------------:|:-----------------------|:-------------|:-------------|--------------------------------:|
> | **266783**                   | -5838.82               | 5838.82      | 3113.89      | 2                               |
> | **266784**                   |  442                   | 4279.66      | 5694.07      | 3                               |
> | **266785**                   | -6828.9                | 6911.77      | 21613.8      | 3                               |
> | **266788**                   |  1312.74               | 1927.12      | 6092.97      | 3                               |
> | **266794**                   | -135,915               | 4610.06      | 27981.9      | 4                               |

* **(a)** Perform a `groupby` operation to calculate, for each customer, the following statistics based on the quantity of items purchased in a transaction (**`qty`** column):
> - The maximum quantity.
> - The minimum quantity.
> - The median quantity.
>
>To achieve this, you should filter the transactions where the quantity is negative using conditional indexing (`qty[qty > 0]`) within a `lambda` function.


In [127]:
customer_stats = transactions[transactions['qty'] > 0].groupby('cust_id')['qty'].agg(
    min_qty=('min'), 
    max_qty=('max'), 
    median_qty=('median')
).reset_index()

print(customer_stats)

      cust_id  min_qty  max_qty  median_qty
0      266783        1        4         2.5
1      266784        2        5         3.0
2      266785        2        5         5.0
3      266788        1        4         1.5
4      266794        1        4         3.0
...       ...      ...      ...         ...
5497   275257        1        5         3.0
5498   275261        1        5         3.0
5499   275262        2        3         2.5
5500   275264        4        5         4.5
5501   275265        1        3         3.0

[5502 rows x 4 columns]


In [ ]:
transactions[transactions['qty']>0].groupby('cust_id').agg({'qty': ['min', 'max', 'median']})

qty           
        min max median
cust_id               
266783    1   4    2.5
266784    2   5    3.0
266785    2   5    5.0
266788    1   4    1.5
266794    1   4    3.0
...      ..  ..    ...
275257    1   5    3.0
275261    1   5    3.0
275262    2   3    2.5
275264    4   5    4.5
275265    1   3    3.0

[5502 rows x 3 columns]

In [115]:
import numpy as np

In [116]:
transactions.groupby('cust_id').agg({'total_amt': ['min', 'max', 'sum'],
                                     'store_type': lambda st: len(np.unique(st))})

total_amt                      store_type
              min       max        sum   <lambda>
cust_id                                          
266783  -5838.820  5838.820   3113.890          2
266784    442.000  4279.665   5694.065          3
266785  -6828.900  6911.775  21613.800          3
266788   1312.740  1927.120   6092.970          3
266794   -135.915  4610.060  27981.915          4
...           ...       ...        ...        ...
275257  -1150.305  7657.650  12574.900          2
275261  -2857.530  2857.530    442.000          3
275262   1750.320  3328.260   5078.580          2
275264   1221.025  2594.540   3815.565          2
275265    245.310  2383.485   3252.015          3

[5502 rows x 4 columns]

In [112]:
boats.groupby('color')['n_reservations'].mean()

color
blue      37.5
green     13.0
red       20.0
yellow    12.0
Name: n_reservations, dtype: float64


> Another way of grouping and summarizing data is to use the `crosstab` function of `pandas` which, as its name suggests, is used to crosstab the data in the columns of a `DataFrame`.
>
> A crosstab allows us to visualize the **appearance frequency** of **pairs of categories** in a `DataFrame`.
>
> <span style="color:#09b038; text-decoration : underline"> Example :</span><br> 
>
> In the `transactions` `DataFrame`, we want to know which are the most frequent category and subcategory pairs (`prod_cat_code` and `prod_subcat_code` columns)
>
> The `crosstab` function of pandas gives us this result:
> 
> ```python
> column1 = transactions['prod_cat_code']
> column2 = transactions['prod_subcat_code']
> pd.crosstab(column1, column2)
> ```
> 
> This instruction produces the following `DataFrame`:
>
> |prod_subcat_code <br><br> prod_cat_code|-1 | 1  | 2  | 3  | 4  | 5 | 6 | 7  | 8 | 9 | 10 | 11 | 12 |
> |--------------------------------------:|--:|---:|---:|---:|---:|--:|--:|---:|--:|--:|---:|---:|---:|
> | **1**                                 |4  |1001|0   |981 |958 |0  |0  |0   |0  |0  |0   |0   |0   |
> | **2**                                 |4  |934 |0   |1040|1005|0  |0  |0   |0  |0  |0   |0   |0   |
> | **3**                                 |11 |0   |0   |0   |1020|950|0  |0   |966|976|945 |0   |0   |
> | **4**                                 |5  |993 |0   |0   |988 |0  |0  |0   |0  |0  |0   |0   |0   |
> | **5**                                 |3  |0   |0   |1023|0   |0  |984|1037|0  |0  |998 |1029|962 |
> | **6**                                 |5  |0   |1002|0   |0   |0  |0  |0   |0  |0  |1025|1013|1057|
> 
> The `(i, j)` cell of the resulting `DataFrame` contains the number of rows of the `DataFrame` having the category `i` for column 1 and the category `j` for column 2.
>
> Thus, it is easy to determine, for example, that **the dominant subcategories** of the category **`4`** are `1` and `4`.
> 
> 
> The **`normalize`** argument of `crosstab` allows to display frequencies as a percentage.
>
> Thus, the argument **`normalize = 1`** normalizes the table over the axis 1 of the crosstab, i.e. its **columns**:
>
> ```python
> #We recover the year of the transaction.
> column1 = transactions['tran_date'].apply(lambda x: x.split('-')[2]).astype(int)
> 
> column2 = transactions['store_type']
> 
> pd.crosstab(column1,
>             column2,
>             normalize = 1)
> ```
>
> This produces the following `DataFrame`:
> 
> | <br><br> store_type <br><br> tran_date | Flagship store | MBR       | TeleShop  | e-Shop    |
> |---------------------------------------:|---------------:|----------:|----------:|----------:|
> | **2011**                               | 0.291942       | 0.323173  | 0.283699  | 0.306947  |
> | **2012**                               | 0.331792       | 0.322093  | 0.336767  | 0.322886  |
> | **2013**                               | 0.335975       | 0.3115    | 0.332512  | 0.320194  |
> | **2014**                               | 0.0402906      | 0.0432339 | 0.0470219 | 0.0499731 |
>
> This `DataFrame` allows us to say that **33.5975%** of the transactions made in a **`'Flagship store'`** took place in **2013**.
>
> Conversely, by entering the argument **`normalize = 0`**, the crosstab is normalized over each **row**:
> 
> | <br><br> store_type <br><br> tran_date | Flagship store | MBR      | TeleShop | e-Shop   |
> |---------------------------------------:|---------------:|---------:|---------:|---------:|
> |**2011**                                | 0.191121       | 0.21548  | 0.182617 | 0.410781 |
> |**2012**                                | 0.20096        | 0.198693 | 0.20056  | 0.399787 |
> |**2013**                                | 0.205522       | 0.194074 | 0.2      | 0.400404 |
> |**2014**                                | 0.173132       | 0.189215 | 0.198675 | 0.438978 |
>
> By normalizing over the rows, we find out that the transactions made in an **`'e-Shop'`** account for **41.0781%** of the transactions of the year **2011**.
>
>
>In the `covid_tests.csv` file, we have a dataset of 200 COVID-19 tests. The columns of this dataset are as follows:
>
>>* `'patient_id'`: ID of the patient tested.
>>* `'test_result'`: Result of the test. Equals 1 if the patient is tested positive and 0 otherwise.
>>* `'infected'`: Equals `1` if the patient was actually infected and `0` otherwise.


* **(b)** Load the dataset contained in the `covid_tests.csv` file. The values are separated by the character `';'`.


* **(c)** Use the `pd.crosstab` function to determine the number of **False Negatives** produced by this test. (A false negative occurs when the test determines that the patient is not infected when they actually are.)


* **(d)** What is the false positive rate of the test? The false positive rate is the **proportion** of false positives in relation to the number of people that are not infected. (A false positive occurs when the test determines that the patient is infected when they are not.)



In [128]:
covid_test = pd.read_csv('covid_tests.csv', sep=';')

In [129]:
covid_test.head()

,patient_id,test_result,infected
0,b508c9a4,1,1
1,b508c9a5,1,1
2,b508c9a6,1,1
3,b508c9a7,1,1
4,b508c9a8,1,1


In [ ]:
pd.crosstab(covid_test['test_result'], covid_test['infected'])

infected,0,1
test_result,,
0,119,3
1,7,71


In [131]:
pd.crosstab(covid_test['test_result'],covid_test['infected'], normalize=1)

infected,0,1
test_result,,
0,0.944444,0.040541
1,0.055556,0.959459


In [ ]:
# The false positive rate is about 5,6%
# 94,4% of healthy people are true negatives